# 実用化のために定期実行スクレイピング（jupyter以外の実行ファイル）

目次：https://docs.google.com/document/d/1MDYw7U1n500t8k5H8tR9PllFRzXOy5KXEyRRWRpIgIk/edit




必要なパッケージは以下の7つです。

- requests
- BeautifulSoup4
- pandas
- gspread
- google-auth
- set_with_dataframe
- schedule

## 目的

これまで、pythonをjupyternotebookの環境から動かしていました。  
ここで、pythonをアプリケーションとして実行する方法を定期実行を通じて学びます。  
これはのちに扱うstreamlitなどに役立ってきます。

## デバックや分析のjupyterファイルと実行ファイルの違い

.ipynbファイルのようなjupyterはコード実行できるメモのような役割をしますが、.pyファイルはプラグラムを実行するのに特化したファイルです。

1. ファイルの概要
    - ipynbファイル: Jupyter Notebookというツールで利用されるファイル形式です。フォーマットとしてはJSONで、markdown方式と組み合わせる事も可能。インタラクティブなコード実行やテキスト、画像などの要素を1つのドキュメントで扱えるため、データ分析や機械学習などの分野でよく使われます。
    - pyファイル: 一般的なPythonスクリプトファイルで、テキストエディターやIDEで編集・実行できます。Pythonプログラムを書く際の標準的なファイル形式です。

2. 歴史的背景
    - ipynbファイル: Jupyter Notebookは、2014年に登場しました。それ以前には、IPython Notebookという名前で知られていました。  
    - pyファイル: Python言語自体が1991年に登場して以来、pyファイルはPythonスクリプトの標準的な形式として使われています。

3. 特徴  
    - ipynbファイル: セルという単位でコードやテキストを分割でき、セル単位で実行や編集が可能です。また、コードの実行結果をその場で表示できるため、プログラムの進行状況を視覚的に把握しやすいです。
    - pyファイル: テキストベースのファイル形式で、複数のPythonコードを一つのファイルにまとめて記述できます。また、他のプログラムやモジュールとしてインポートして利用することができます。

4. テスト環境や本番環境での取り扱い
    - ipynbファイル: 主にテストや実験、データ分析のための環境で利用されます。本番環境では、ipynbファイルをpyファイルに変換して実行することが一般的です。
    - pyファイル: テスト環境でも本番環境でも利用されます。モジュール化が容易であるため、再利用性やメンテナンス性に優れています。

ipynbファイルはインタラクティブな環境でのコードの実行や解析、可視化に向いている一方で、pyファイルはより一般的なプログラミングや本番環境での運用に適しています。どちらのファイル形式も、それぞれの目的に応じて適切に選択・利用することが重要です。

初学者にとっては、以下のような使い分けが参考になります。

・ipynbファイル (Jupyter Notebook):
データの探索や可視化、機械学習のモデル構築・評価、教育用途やプレゼンテーションなど、途中経過を確認しながら進める作業に向いています。

・pyファイル (Pythonスクリプト):
一般的なプログラミング、ライブラリやフレームワークの開発、アプリケーションの本番環境での運用など、再利用性や拡張性が重要な場面で利用されます。

これらの違いを理解して、自分のプロジェクトや学習目的に応じて適切なファイル形式を選択することが、効果的なPythonプログラミングにつながります。

## 全体像

前章で習ったPythonのスクレイピングのコードを用いて、毎日スクレイピングを実行してgoogleスプレッドシートに取得したデータを格納します。  
今回はこの定期実行アプリを作ります。

### 使うライブラリ

- gspread
- Credentials
- set_with_dataframe
3つのライブラリを使って、スプレッドシートにデータを格納します。

これらのライブラリはGoogle Cloud Platform（GCP）から提供されているAPIと連携しているため、GCPアカウントが必須です。

In [ ]:
# それぞれのライブラリインストール方法です！
!pip install gspread
!pip install google-auth
!pip install gspread-dataframe
!pip install schedule

#### GCPの設定方法

>アカウント作成

googleが提供するスプレッドシートにアクセスするためにgoogleから提供されるAPIとライブラリを使う必要があります。  
アカウントが必要なので登録しましょう！  
https://cloud.google.com/

>使用する機能設定や使用権限設定をAPIキーとして発行する

Google Cloud Platformを使用してGoogleスプレッドシートのAPIを扱うためには、以下の手順になります。

1. GCPプロジェクトの作成:
    - 1つのアプリのために複数のAPIを使用する想定でプロジェクトというものを作成する必要があります。  
    ここで作成するプロジェクト名は”はじめてのスプレッドシートAPI”などわかりやすいものにしておきましょう。
    - 作成方法はGoogle Cloud Console（https://console.cloud.google.com）にアクセスします。  
    - プロジェクトの選択ボックスで新しいプロジェクトを作成します。
1. Google Sheets APIを有効にする:

    - プロジェクトダッシュボードで、「APIとサービス」→「ライブラリ」を選択します。検索ボックスに「Google Sheets API」と入力し、該当するAPIを見つけて有効化します。
1. 認証情報を作成する:

    - プロジェクトダッシュボードで、「APIとサービス」→「認証情報」を選択します。
    - 「認証情報を作成」ボタンをクリックし、サービスアカウントキーを作成します。
    - サービスアカウントのメールアドレス、ロール（スプレッドシートへのアクセス権限を持つ必要があります）を設定し、JSON形式の鍵ファイルをダウンロードします。
    - このダウンロードしたサービスアカウントがAPIを使用するためのAPIキーになります。前章で使用した楽天APIを使用するためのAPIキーのような認証キーが複数記載されたファイルになります。
1. Google Sheets APIを使用するアプリケーションに認証情報を追加する:

    - Googleスプレッドシートを操作するアプリケーションに認証情報を追加する必要があります。
    認証情報（JSONファイル）をこのjupyter notebookがある場所に移動させてください。
    - 楽天APIの場合はAPIキーを直接入力しましたが、この認証情報はファイルを読み込んで、APIキーを読み込みます。

>使い方

以下の手順でこの機能を使います。

1. 拡張機能であるライブラリを読み込む
1. APIの使用権限 認証のための準備
1. 準備した認証設定コードを使ってスプレッドシートにアクセス
1. スプレッドシートからデータ読み込む
1. 読み込んだデータを可視化のためにpandasを使って構造化データにする

サンプルとしてこちらが用意したスプレッドシートにアクセスしてデータを取得します。  
https://docs.google.com/spreadsheets/d/1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24


In [1]:
# ここから拡張機能であるライブラリを読み込む

import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート


# ここからAPIの使用権限 認証のための準備

# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'



# ここから準備した認証設定コードを使ってスプレッドシートにアクセス

# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(SP_SHEET_KEY)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'sample'


# ここからスプレッドシートからデータ読み込む

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)
data = worksheet.get_all_values() # スプレッドシートにある既存のデータをdataに代入


# ここは読み込んだデータを可視化のためにpandasを使って構造化データにする

df_old = pd.DataFrame(data[1:], columns=data[0]) # スプレッドシートにある既存のデータをデータフレームに格納し、df_oldに代入

### コーディングのフロー

- 今から行うタスクの切り分け
    - スクレイピングでサイトにアクセス
    - スクレイピングでデータ取得
    - 取得データをスプレッドシートに格納
    - このフローを定期実行する
- やることの流れ
    - jupyterでスクレイピングの動作を確認
    - スプレッドシートにアクセス
    - スクレイピングしたコードをスプレッドシートに格納
    - 定期実行
    - スクレイピングコードを定期実行アプリへ
- スクレイピングのコードをpythonの実行ファイルへ
    - 動作確認できたipynbコードを実行ファイルであるpyファイルへ

## 説明のためのコード

まず全体像のコードを確認して、その後にそれぞれのコードについて確認してきます。

>全体のコード

全体の流れは以下のようになります。
1. ライブラリをインポートして、それぞれの拡張機能を扱えるようにする
1. 定期実行したいコードが書かれた関数を読み込み
    - 定期実行の中身
        1. 前回の章で作成したスクレイピングのコードでデータ取得
        1. APIキーを使ってスプレッドシートにアクセスするための準備
        1. スプレッドシートにアクセスして既存のデータを取得
        1. スクレイピングで取得したデータとスプレッドシートにある既存のデータを結合
        1. 結合した最新データをスプレッドシートに書き込み
1. 実行コードでスケジュールに定期実行したい関数を登録
1. コードが終了しないようにwhile構文を使って終了を防止して、スケジュールの実行を待つ

以下がコードの全体像になります。

In [ ]:
# ここから張機能をインポート

import requests # リクエスト機能をインポート
from bs4 import BeautifulSoup # html解析機能をインポート
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート
import time # 実行待機のための機能をインポート
import schedule # 定期実行するための機能をインポート


# ここから定期実行する関数の内容
def job():


    # ここから前章のスクレイピングのコード
    REQUEST_URL = 'https://travel.rakuten.co.jp/yado/okinawa/nahashi.html' # リクエストのアクセス先をREQUEST_URLに代入
    res = requests.get(REQUEST_URL) # 楽天トラベルにアクセスし、そのデータをresに代入
    res.encoding = 'utf-8' # 文字化けしないように文字コードをutf-8に指定
    soup = BeautifulSoup(res.text,"html.parser") # 取得したhtmlをBeautifulSoupで解析して、soupに代入
    hotel_section_from_html = soup.select('section')  # すべてのsectionタグを取得します。

    #すべてのsctionのデータを持つhotel_section_from_htmlから欲しい情報を取得出来る事条件を作ります。
    # 条件でselect_one('p.area')にデータを含まない場合、不要なsectionなので条件文で排除できるように準備します。

    # すべてのsctionのデータを持つhotel_section_from_htmlをhsに1つずつsectionデータを代入して実行します。
    hotel_section = []
    for hs in hotel_section_from_html:
        a = hs.select_one('p.area')
        if (a != None):
            hotel_section.append(hs) # aに情報がある時に実行したいので、条件はNoneではないというa != Noneになります。

    hotelName = [] # ホテル名を格納する空配列を用意します。
    hotelMinCharge = [] # ホテルの料金を格納する空配列を用意します。
    reviewAverage = [] # ホテル評価を格納する空配列を用意します。
    hotel_locate = [] # ホテル住所を格納する空配列を用意します。

    # hotel_sectionからsectionを1つずつ取り出してhsに代入して実行します。
    for hs in hotel_section:
        hs1 = hs.select_one('h1 a').text #ホテル名をhs1に代入
        if (hs.select_one('p.htlPrice span') != None):
            hs2 = hs.select_one('p.htlPrice span').text.replace("円〜","").replace(",","") # 値段をhs2に代入
        else:
            hs2 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。
        hs3 = hs.select_one('p.cstmrEvl strong').text # 評価をhs3に代入
        hs4 = hs.select_one('p.htlAccess').text.replace("\n","").replace(" ","").replace("[地図を見る]","").replace("　","") # 住所をhs4に代入
        hotelName.append(hs1) #抽出したホテル名をhotelNameに追加
        hotelMinCharge.append(hs2) #抽出したホテルの料金をhotelMinChargeに追加
        reviewAverage.append(hs3) #抽出したホテルの評価をreviewAverageに追加
        hotel_locate.append(hs4) #抽出したホテル住所をhotel_locateに追加

    # pandasのデータフレームに使うデータを定義します。
    data_list = {
        "hotelName" : hotelName,
        "hotelMinCharge" : hotelMinCharge,
        "reviewAverage" : reviewAverage,
        "hotel_locate" : hotel_locate,
    }

    # 定義したデータをpandasに読み込ませます
    # pd.DataFrame(ここにデータ)でデータフレームに変換できます。
    df = pd.DataFrame(data_list)

    # drop_duplicates()で重複したデータを削除してくれます。
    # drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
    df.drop_duplicates(inplace=True)

    # 列の認識番号であるindexが列の順番と一致していないので、reset_indexで番号を振り直します。
    # reset_index(drop=True,inplace=True)とすることで、元のindexを削除して新たに生成したindexを作成し、元のデータを更新してくれます。
    df.reset_index(drop=True,inplace=True)




    # ここからスプレッドシートに格納するコード

    # 認証のために機能役割を決めるアクセス先をscopesに設定
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]

    # その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
    # 認証キーを使うアクセス先をscopesに代入
    credentials = Credentials.from_service_account_file(
        'service_account.json',
        scopes=scopes
    )

    # 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
    gc = gspread.authorize(credentials)

    # 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
    # https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
    SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'

    # 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
    sh = gc.open_by_key(SP_SHEET_KEY)

    # 参照するシート名をSP_SHEETに代入
    SP_SHEET = 'sample'

    # gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
    worksheet = sh.worksheet(SP_SHEET)

    data = worksheet.get_all_values() # スプレッドシートにある既存のデータをdataに代入
    df_old = pd.DataFrame(data[1:], columns=data[0]) # スプレッドシートにある既存のデータをデータフレームに格納し、df_oldに代入
    df_new = df # スクレイピングで取得した新しいデータdfをdf_newに代入
    df_upload = pd.concat([df_old,df_new]) # 既存のdf_oldとdf_newをpd.concatで結合
    df_upload.reset_index(drop=True,inplace=True)

    # シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
    # を使ってシートにデータフレームのデータを書き込みます。
    set_with_dataframe(sh.worksheet("sample"), df_upload, include_index=False)




# ここから定期実行のタイミングを登録するコード

schedule.clear() # scheduleに実行予定が残っていないように初期化しておきます。
schedule.every().day.do(job) # scheduleに実行予定として１日ごとにjobという関数を実行を追加します。


# ここから定期実行するコード

# whileで繰り返しscheduleの実行予定を確認します。
while True:
    schedule.run_pending() # scheduleに追加されている実行予定をチェックします
    time.sleep(1) # 重くならないように念のため１秒の猶予を設けます。

>それぞれのコードの説明

3つにわけて説明します。
1. （おさらい）スクレイピングの部分
1. スプレッドシートアクセスとデータ取得の部分
1. データ更新の部分
1. 定期実行の部分

1.（おさらい）スクレイピングの部分

前章で扱ったスクレイピングのコードを軽く振り返ってみましょう。

In [ ]:
# ここから前章のスクレイピングのコード
REQUEST_URL = 'https://travel.rakuten.co.jp/yado/okinawa/nahashi.html' # リクエストのアクセス先をREQUEST_URLに代入
res = requests.get(REQUEST_URL) # 楽天トラベルにアクセスし、そのデータをresに代入
res.encoding = 'utf-8' # 文字化けしないように文字コードをutf-8に指定
soup = BeautifulSoup(res.text,"html.parser") # 取得したhtmlをBeautifulSoupで解析して、soupに代入
hotel_section_from_html = soup.select('section')  # すべてのsectionタグを取得します。

#すべてのsctionのデータを持つhotel_section_from_htmlから欲しい情報を取得出来る事条件を作ります。
# 条件でselect_one('p.area')にデータを含まない場合、不要なsectionなので条件文で排除できるように準備します。

# すべてのsctionのデータを持つhotel_section_from_htmlをhsに1つずつsectionデータを代入して実行します。
hotel_section = []
for hs in hotel_section_from_html:
    a = hs.select_one('p.area')
    if (a != None):
        hotel_section.append(hs) # aに情報がある時に実行したいので、条件はNoneではないというa != Noneになります。

hotelName = [] # ホテル名を格納する空配列を用意します。
hotelMinCharge = [] # ホテルの料金を格納する空配列を用意します。
reviewAverage = [] # ホテル評価を格納する空配列を用意します。
hotel_locate = [] # ホテル住所を格納する空配列を用意します。

# hotel_sectionからsectionを1つずつ取り出してhsに代入して実行します。
for hs in hotel_section:
    hs1 = hs.select_one('h1 a').text #ホテル名をhs1に代入
    if (hs.select_one('p.htlPrice span') != None):
        hs2 = hs.select_one('p.htlPrice span').text.replace("円〜","").replace(",","") # 値段をhs2に代入
    else:
        hs2 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。
    hs3 = hs.select_one('p.cstmrEvl strong').text # 評価をhs3に代入
    hs4 = hs.select_one('p.htlAccess').text.replace("\n","").replace(" ","").replace("[地図を見る]","").replace("　","") # 住所をhs4に代入
    hotelName.append(hs1) #抽出したホテル名をhotelNameに追加
    hotelMinCharge.append(hs2) #抽出したホテルの料金をhotelMinChargeに追加
    reviewAverage.append(hs3) #抽出したホテルの評価をreviewAverageに追加
    hotel_locate.append(hs4) #抽出したホテル住所をhotel_locateに追加

# pandasのデータフレームに使うデータを定義します。
data_list = {
    "hotelName" : hotelName,
    "hotelMinCharge" : hotelMinCharge,
    "reviewAverage" : reviewAverage,
    "hotel_locate" : hotel_locate,
}

# 定義したデータをpandasに読み込ませます
# pd.DataFrame(ここにデータ)でデータフレームに変換できます。
df = pd.DataFrame(data_list)

# drop_duplicates()で重複したデータを削除してくれます。
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)

# 列の認識番号であるindexが列の順番と一致していないので、reset_indexで番号を振り直します。
# reset_index(drop=True,inplace=True)とすることで、元のindexを削除して新たに生成したindexを作成し、元のデータを更新してくれます。
df.reset_index(drop=True,inplace=True)

2.スプレッドシートアクセスとデータ取得の部分

googleにどの機能を使うのかを、アクセス先で指定します。  
今回は　https://www.googleapis.com/auth/spreadsheets と https://www.googleapis.com/auth/drive を指定します。

この２つをscopesという変数に格納します。

In [ ]:
# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

アクセス先に渡す認証として、先ほどダウンロードしたAPIキーを使います。  
アクセス先と認証キーをCredentials.from_service_account_fileに設定して、credentialsに代入しておきます。

このcredentialsを使ってgspread.authorize(credentials)で認証をします。  
その認証結果やgoogleのサービスへのアクセス権限をgcに代入しておきます。

In [ ]:
# その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

アクセスしたいスプレッドシートを認証完了した 変数gcで開きます。
その開いた状態データをshに格納します。  


In [ ]:
# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'

# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(SP_SHEET_KEY)

アクセスしたスプレッドシートのどのシートのデータを取得するかを指定します。  
シートを参照した状態のデータをworksheetに代入します。

In [ ]:
# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'sample'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

参照しているシートからget_all_values()で、記載されているすべての値を取得します。  
そのデータをdataに格納します。

In [ ]:
data = worksheet.get_all_values() # スプレッドシートにある既存のデータをdataに代入

エクセルやスプレッドシートのような見た目にするために、取得したデータを構造化データに変換します。  
今回のデータでは、１列目にカラム名、２列目からデータとなっているため、pd.DataFrame(data[1:], columns=data[0])というコードになります。   
既存のデータなので、df_oldとしておきます。

In [ ]:
df_old = pd.DataFrame(data[1:], columns=data[0]) # スプレッドシートにある既存のデータをデータフレームに格納し、df_oldに代入

3.データ更新の部分

スプレッドシートに上書きアップロードするには  
set_with_dataframe(sh.worksheet(ここシート名), ここデータ, include_index=False)   
を使います。

pandasの行数であるindexは必要ないので、include_index=Falseになります。

In [ ]:
# 既存のデータdf_oldと区別するためにdfをdf_newとしておきます

df_new = df # スクレイピングで取得した新しいデータdfをdf_newに代入
df_upload = pd.concat([df_old,df_new]) # 既存のdf_oldとdf_newをpd.concatで結合
df_upload.reset_index(drop=True,inplace=True)

# シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
# を使ってシートにデータフレームのデータを書き込みます。
set_with_dataframe(sh.worksheet("sample"), df_upload, include_index=False)

4.定期実行の部分

見やすいようにスクレイピングとデータ更新の部分を   
task = "ここにスクレイピングの実行内容がきます。"   
としています。

schedule.every(2).seconds.do(job) で、実行タイミングを登録して、   
while構文の中で、実行タイミングを待ちます。

実行タイミングの登録は複数登録することが可能です。  
逆に言えば、schedule.clear()でリセットしておかないと、複数買い登録されて、複数回実行する可能性もあるので気をつけましょう。

In [ ]:
import time # 実行待機のための機能をインポート
import schedule # 定期実行するための機能をインポート



# jobという関数を定義しておきます。ここが定期実行で呼び出される処理になります。
def job():
    task = "ここにスクレイピングの実行内容とデータ更新のコードがきます。" # 処理を仮置きしておきます


schedule.clear() # scheduleに実行予定が残っていないように初期化しておきます。
schedule.every(2).seconds.do(job) # scheduleに実行予定として２秒ごとにjobという関数を実行を追加します。

# whileで繰り返しscheduleの実行予定を確認します。
while True:
    schedule.run_pending() # scheduleに追加されている実行予定をチェックします
    time.sleep(1) # 重くならないように念のため１秒の猶予を設けます。

## 実際のコーディング

学習する内容の流れは以下のようになります

1. スプレッドシートからデータ取得と書き込み
    1. 講義用に用意されたスプレッドシートにアクセスする
    1. 講義用に用意されたスプレッドシートからデータを取得
    1. 取得したデータを受講生自身が用意したスプレッドシートにデータをアップロード
1. データ更新
    1. 講義用のスプレッドシートのデータを追加したい仮データとして、受講生のスプレッドシートにさらに追加
    1. 仮データではなく、スクレイピングしたデータをスプレッドシートに追加
1. 定期実行
    1. 定期実行のための基礎
    1. スクレイピング+データ更新を定期実行化する

### スプレッドシートからデータ取得と書き込み

取得したAPIキーでスプレッドシートにアクセスしてみましょう  
サンプルとしてこちらが用意したシートにアクセスしてみましょう。
- https://docs.google.com/spreadsheets/d/1nXkj9QxjBt-P0lshNKHXtnb-6nSAIHarb9_THQcbEF0/edit#gid=0  
#登録して取得したservice_account.jsonをサンプルとして配置しているservice_account.jsonと入れ替えましょう！

In [1]:
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート

# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
SP_SHEET_KEY = '1nXkj9QxjBt-P0lshNKHXtnb-6nSAIHarb9_THQcbEF0'

# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(SP_SHEET_KEY)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'sample'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

# 得られたsampleシートの情報をget_all_values()で値をすべて取得してdataに代入
data = worksheet.get_all_values()


In [53]:
# df_sampleにsampleシートから得られたデータをpandasのDataFrameを使って代入してみましょう
# sampleシートは0行目はカラム名なので、columns=data[0]、データがあるのは２列目からなのでdata[1:]をデータフレームに代入します。
df_sample = pd.DataFrame(data[1:], columns=data[0])
df_sample

,hotelName,hotelMinCharge,reviewAverage,hotel_locate
0,ホテルグレイスリー那覇,4273,4.48,〒900-0014沖縄県那覇市松尾1-3-6
1,那覇ウエスト・イン,2268,4.36,〒900-0036沖縄県那覇市西1-16-7
2,沖縄ハーバービューホテル,3069,4.18,〒900-0021沖縄県那覇市泉崎2-46
3,ホテルグランコンソルト那覇（２０２２年９月開業）,4546,4.56,〒900-0014沖縄県那覇市松尾1-18-25
4,ダブルツリーｂｙヒルトン那覇首里城,4831,3.33,〒903-8601沖縄県那覇市首里山川町1-132-1
5,アルモントホテル那覇県庁前,4410,4.48,〒900-0015沖縄県那覇市久茂地1-3-5
6,ダイワロイネットホテル那覇おもろまち,3319,4.38,〒900-0006沖縄県那覇市おもろまち1-1-12
7,東急ステイ沖縄那覇,4273,4.59,〒900-0025沖縄県那覇市壷川3-2-1
8,ダイワロイネットホテル沖縄県庁前,3128,4.24,〒900-0021沖縄県那覇市泉崎1-11-2
9,ホテルアクアチッタナハ,4394,4.42,〒900-0016沖縄県那覇市前島3-2-20


自分のスプレッドシート環境に取得したdf_sampleをアップロードしてみましょう！
シート名はsampleにしておきましょう。SP_SHEET_KEYに使う文字列は
https://docs.google.com/spreadsheets/d/「ここの部分」になります。

この資料では1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24にアップロードしていますが、ご自身のスプレッドシート環境で大丈夫です。

In [66]:
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート

# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
# 認証キーを使うアクセス先をscopesに代入
credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'

# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(SP_SHEET_KEY)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'sample'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

In [ ]:
# シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
# を使ってシートにデータフレームのデータを書き込みます。
set_with_dataframe(sh.worksheet("sample"), df_sample, include_index=False)

### データ更新

スプレッドシートに更新できたので、追加データを更新をしてみましょう！  
楽天トラベルからの追加取得データの代わりにdf_sampleを疑似追加取得データとして使います。

In [67]:
data = worksheet.get_all_values() # ご自身のスプレッドシートに書き込んだデータをdataに代入します。

In [ ]:
# 既存のデータとしてdf_oldに書き込んだデータを代入します。
# 新規データとしてdf_newにdf_sampleのデータをコピーしたデータdf_sample.copy()を代入します。

df_old = pd.DataFrame(data[1:], columns=data[0])
df_new = df_sample.copy() # df_sampleと同じデータを持ったdf_sampleと関係を持たないデータをcopy()で作成できます。

In [57]:
# 書き込むデータを変数df_uploadに代入する。
# 内容はdf_oldとdf_newを結合したデータ
# 方法は pd.concat([結合したいデータ1,結合したいデータ2])
df_upload = pd.concat([df_old,df_new])

In [58]:
# 結合できているかどうかアップロードする内容を確認します
display(df_upload)

,hotelName,hotelMinCharge,reviewAverage,hotel_locate
0,ホテルグレイスリー那覇,4273,4.48,〒900-0014沖縄県那覇市松尾1-3-6
1,那覇ウエスト・イン,2268,4.36,〒900-0036沖縄県那覇市西1-16-7
2,沖縄ハーバービューホテル,3069,4.18,〒900-0021沖縄県那覇市泉崎2-46
3,ホテルグランコンソルト那覇（２０２２年９月開業）,4546,4.56,〒900-0014沖縄県那覇市松尾1-18-25
4,ダブルツリーｂｙヒルトン那覇首里城,4831,3.33,〒903-8601沖縄県那覇市首里山川町1-132-1
...,...,...,...,...
30,ノボテル沖縄那覇,3924,4.34,〒902-0062沖縄県那覇市松川40番地
31,ハイアットリージェンシー那覇沖縄,8000,4.65,〒900-0013沖縄県那覇市牧志3-6-20
32,ホテルコレクティブ,7564,4.77,〒900-0014沖縄県那覇市松尾2-5-7
33,コンフォートイン那覇泊港,3682,4.39,〒900-0016沖縄県那覇市前島3-1-4


indexが適当でないので、振り直しましょう

In [61]:
# reset_index(drop=True,inplace=True)で既存のindexを削除して、振り直したindex番号を追加します。
df_upload.reset_index(drop=True,inplace=True)
display(df_upload) # 確認

,hotelName,hotelMinCharge,reviewAverage,hotel_locate
0,ホテルグレイスリー那覇,4273,4.48,〒900-0014沖縄県那覇市松尾1-3-6
1,那覇ウエスト・イン,2268,4.36,〒900-0036沖縄県那覇市西1-16-7
2,沖縄ハーバービューホテル,3069,4.18,〒900-0021沖縄県那覇市泉崎2-46
3,ホテルグランコンソルト那覇（２０２２年９月開業）,4546,4.56,〒900-0014沖縄県那覇市松尾1-18-25
4,ダブルツリーｂｙヒルトン那覇首里城,4831,3.33,〒903-8601沖縄県那覇市首里山川町1-132-1
...,...,...,...,...
65,ノボテル沖縄那覇,3924,4.34,〒902-0062沖縄県那覇市松川40番地
66,ハイアットリージェンシー那覇沖縄,8000,4.65,〒900-0013沖縄県那覇市牧志3-6-20
67,ホテルコレクティブ,7564,4.77,〒900-0014沖縄県那覇市松尾2-5-7
68,コンフォートイン那覇泊港,3682,4.39,〒900-0016沖縄県那覇市前島3-1-4


スプレッドシートに更新してみましょう！

In [69]:
# シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
# を使ってシートにデータフレームのデータを書き込みます。
set_with_dataframe(sh.worksheet("sample"), df_upload, include_index=False)

既存のデータにdf_sampleデータを追加して更新することが出来たので、
次にスクレイピングで取得したデータで更新してみましょう

スクレイピングのコードを整理して、データを取得しましょう

In [62]:
import requests # リクエスト機能をインポート
from bs4 import BeautifulSoup # html解析機能をインポート
import pandas as pd # データフレームを扱うpandasをインポート

REQUEST_URL = 'https://travel.rakuten.co.jp/yado/okinawa/nahashi.html' # リクエストのアクセス先をREQUEST_URLに代入
res = requests.get(REQUEST_URL) # 楽天トラベルにアクセスし、そのデータをresに代入
res.encoding = 'utf-8' # 文字化けしないように文字コードをutf-8に指定
soup = BeautifulSoup(res.text,"html.parser") # 取得したhtmlをBeautifulSoupで解析して、soupに代入
hotel_section_from_html = soup.select('section')  # すべてのsectionタグを取得します。

#すべてのsctionのデータを持つhotel_section_from_htmlから欲しい情報を取得出来る事条件を作ります。
# 条件でselect_one('p.area')にデータを含まない場合、不要なsectionなので条件文で排除できるように準備します。

# すべてのsctionのデータを持つhotel_section_from_htmlをhsに1つずつsectionデータを代入して実行します。
hotel_section = []
for hs in hotel_section_from_html:
    a = hs.select_one('p.area')
    if (a != None):
        hotel_section.append(hs) # aに情報がある時に実行したいので、条件はNoneではないというa != Noneになります。

hotelName = [] # ホテル名を格納する空配列を用意します。
hotelMinCharge = [] # ホテルの料金を格納する空配列を用意します。
reviewAverage = [] # ホテル評価を格納する空配列を用意します。
hotel_locate = [] # ホテル住所を格納する空配列を用意します。

# hotel_sectionからsectionを1つずつ取り出してhsに代入して実行します。
for hs in hotel_section:
    hs1 = hs.select_one('h1 a').text #ホテル名をhs1に代入
    if (hs.select_one('p.htlPrice span') != None):
        hs2 = hs.select_one('p.htlPrice span').text.replace("円〜","").replace(",","") # 値段をhs2に代入
    else:
        hs2 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。
    hs3 = hs.select_one('p.cstmrEvl strong').text # 評価をhs3に代入
    hs4 = hs.select_one('p.htlAccess').text.replace("\n","").replace(" ","").replace("[地図を見る]","").replace("　","") # 住所をhs4に代入
    hotelName.append(hs1) #抽出したホテル名をhotelNameに追加
    hotelMinCharge.append(hs2) #抽出したホテルの料金をhotelMinChargeに追加
    reviewAverage.append(hs3) #抽出したホテルの評価をreviewAverageに追加
    hotel_locate.append(hs4) #抽出したホテル住所をhotel_locateに追加

# pandasのデータフレームに使うデータを定義します。
data_list = {
    "hotelName" : hotelName,
    "hotelMinCharge" : hotelMinCharge,
    "reviewAverage" : reviewAverage,
    "hotel_locate" : hotel_locate,
}

# 定義したデータをpandasに読み込ませます
# pd.DataFrame(ここにデータ)でデータフレームに変換できます。
df = pd.DataFrame(data_list)

# drop_duplicates()で重複したデータを削除してくれます。
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)

# 列の認識番号であるindexが列の順番と一致していないので、reset_indexで番号を振り直します。
# reset_index(drop=True,inplace=True)とすることで、元のindexを削除して新たに生成したindexを作成し、元のデータを更新してくれます。
df.reset_index(drop=True,inplace=True)

正常に動作しているかどうか、dfの中身をチェックしてみましょう！

In [64]:
display(df)

,hotelName,hotelMinCharge,reviewAverage,hotel_locate
0,ホテルグレイスリー那覇,4273,4.48,〒900-0014沖縄県那覇市松尾1-3-6
1,那覇ウエスト・イン,2268,4.36,〒900-0036沖縄県那覇市西1-16-7
2,沖縄ハーバービューホテル,3069,4.18,〒900-0021沖縄県那覇市泉崎2-46
3,ホテルグランコンソルト那覇（２０２２年９月開業）,4546,4.56,〒900-0014沖縄県那覇市松尾1-18-25
4,ダブルツリーｂｙヒルトン那覇首里城,4831,3.33,〒903-8601沖縄県那覇市首里山川町1-132-1
5,アルモントホテル那覇県庁前,4410,4.48,〒900-0015沖縄県那覇市久茂地1-3-5
6,ダイワロイネットホテル那覇おもろまち,3319,4.38,〒900-0006沖縄県那覇市おもろまち1-1-12
7,東急ステイ沖縄那覇,4273,4.59,〒900-0025沖縄県那覇市壷川3-2-1
8,ダイワロイネットホテル沖縄県庁前,3128,4.24,〒900-0021沖縄県那覇市泉崎1-11-2
9,ホテルアクアチッタナハ,4394,4.42,〒900-0016沖縄県那覇市前島3-2-20


取得と更新のコードを整理して、スクレイピング後にスプレッドシートに更新するようにしてみましょう！

In [72]:
#先ほどのスクレイピングコード

import requests
from bs4 import BeautifulSoup
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート

REQUEST_URL = 'https://travel.rakuten.co.jp/yado/okinawa/nahashi.html'
res = requests.get(REQUEST_URL)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text,"html.parser")
hotel_section_from_html = soup.select('section')

hotel_section = []
for hs in hotel_section_from_html:
    a = hs.select_one('p.area')
    if (a != None):
        hotel_section.append(hs)

hotelName = []
hotelMinCharge = []
reviewAverage = []
hotel_locate = []
for hs in hotel_section:
    hs1 = hs.select_one('h1 a').text
    if (hs.select_one('p.htlPrice span') != None):
        hs2 = hs.select_one('p.htlPrice span').text.replace("円〜","").replace(",","")
    else:
        hs2 = -1
    hs3 = hs.select_one('p.cstmrEvl strong').text
    hs4 = hs.select_one('p.htlAccess').text.replace("\n","").replace(" ","").replace("[地図を見る]","").replace("　","")
    hotelName.append(hs1)
    hotelMinCharge.append(hs2)
    reviewAverage.append(hs3)
    hotel_locate.append(hs4)

data_list = {
    "hotelName" : hotelName,
    "hotelMinCharge" : hotelMinCharge,
    "reviewAverage" : reviewAverage,
    "hotel_locate" : hotel_locate,
}

df = pd.DataFrame(data_list)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

#ここまでスクレイピング

#ここから追加



# 認証のために機能役割を決めるアクセス先をscopesに設定
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

# 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
gc = gspread.authorize(credentials)

# 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
# https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'

# 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
sh = gc.open_by_key(SP_SHEET_KEY)

# 参照するシート名をSP_SHEETに代入
SP_SHEET = 'sample'

# gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
worksheet = sh.worksheet(SP_SHEET)

data = worksheet.get_all_values() # スプレッドシートにある既存のデータをdataに代入
df_old = pd.DataFrame(data[1:], columns=data[0]) # スプレッドシートにある既存のデータをデータフレームに格納し、df_oldに代入
df_new = df # スクレイピングで取得した新しいデータdfをdf_newに代入
df_upload = pd.concat([df_old,df_new]) # 既存のdf_oldとdf_newをpd.concatで結合
df_upload.reset_index(drop=True,inplace=True) # 行に振られたindexが綺麗に順番通り出ないのでを振り直す

# シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
# を使ってシートにデータフレームのデータを書き込みます。
set_with_dataframe(sh.worksheet("sample"), df_upload, include_index=False)

### 定期実行

次に定期実行をするためにコードを実行を待つという機能が必要なので、timeという遅延ライブラリを使ってみましょう

In [11]:
import time # 実行待機のための機能をインポート
import datetime # 現在時刻を取得する機能をインポート

# ２秒処理を待つコードで待機しているか確認します。
# 現在時刻はdatetime.datetime.now()で取得できます。
# 処理待機はtime.sleep(ここに秒数の数字)でできます
print(datetime.datetime.now())
time.sleep(2)
print(datetime.datetime.now())

2023-03-06 09:28:25.953542
2023-03-06 09:28:27.955372


定期実行のコードを触る前に定期実行を中断できるようにjupyterでの実行停止方法も確認しておきましょう。  
確認方法としてtime.sleep(10)の実行を停止してみましょう。
vscodeの方は四角ボタンで停止できます。

In [12]:
print("開始")
time.sleep(10) # 実行後、コード処理が１０秒待機している間に処理を中断してみましょう。
print("中断出来ませんでした")

KeyboardInterrupt: 

scheduleを活用して2秒毎の定期実行をしてみましょう  
scheduleの実行条件は実行するたびに追加されるので、重複を回避するために必ずschedule.clear()で初期化するようにしましょう。

In [25]:
import time # 実行待機のための機能をインポート
import datetime # 現在時刻を取得する機能をインポート
import schedule # 定期実行するための機能をインポート

# jobという関数を定義しておきます。
def job():
    task = "ここにスクレイピングの実行内容がきます。" # 処理を仮置きしておきます
    print(datetime.datetime.now()) # 定期実行できているか現在時刻を表示させます。

schedule.clear() # scheduleに実行予定が残っていないように初期化しておきます。
schedule.every(2).seconds.do(job) # scheduleに実行予定として２秒ごとにjobという関数を実行を追加します。

# whileで繰り返しscheduleの実行予定を確認します。
while True:
    schedule.run_pending() # scheduleに追加されている実行予定をチェックします
    time.sleep(1) # 重くならないように念のため１秒の猶予を設けます。

2023-03-06 09:47:23.658681
2023-03-06 09:47:26.660682
2023-03-06 09:47:29.674705
2023-03-06 09:47:32.679285


KeyboardInterrupt: 

他の定期実行の条件の具体例はこのようになります！
- 3分おきに実行
    - schedule.every(3).minutes.do(job)
- 1時間おきに実行
    - schedule.every().hour.do(job)
- 毎日10:30
    - schedule.every().day.at("10:30").do(job)
- 毎週月曜
    - schedule.every().monday.do(job)
- 毎週月曜9:00
    - schedule.every().monday.at("9:00").do(job)

app.pyファイルに毎日スクレイピングしてスプレッドシートに更新するアプリにしましょう！

↓講師向けファイル内容↓

In [ ]:
import requests # リクエスト機能をインポート
from bs4 import BeautifulSoup # html解析機能をインポート
import pandas as pd # スプレッドシートから得たデータをデータフレームに変換する機能をインポートし、省略してpdと呼べるようにas pdを付ける
import gspread # スプレッドシートのデータを扱うライブラリをインポート
from google.oauth2.service_account import Credentials # スプレッドシートの認証機能をインポート
from gspread_dataframe import set_with_dataframe # スプレッドシートのデータとpandasライブラリのデータを紐づける機能をインポート
import time # 実行待機のための機能をインポート
import schedule # 定期実行するための機能をインポート


# 定期実行する内容
def job():
    REQUEST_URL = 'https://travel.rakuten.co.jp/yado/okinawa/nahashi.html' # リクエストのアクセス先をREQUEST_URLに代入
    res = requests.get(REQUEST_URL) # 楽天トラベルにアクセスし、そのデータをresに代入
    res.encoding = 'utf-8' # 文字化けしないように文字コードをutf-8に指定
    soup = BeautifulSoup(res.text,"html.parser") # 取得したhtmlをBeautifulSoupで解析して、soupに代入
    hotel_section_from_html = soup.select('section')  # すべてのsectionタグを取得します。

    #すべてのsctionのデータを持つhotel_section_from_htmlから欲しい情報を取得出来る事条件を作ります。
    # 条件でselect_one('p.area')にデータを含まない場合、不要なsectionなので条件文で排除できるように準備します。

    # すべてのsctionのデータを持つhotel_section_from_htmlをhsに1つずつsectionデータを代入して実行します。
    hotel_section = []
    for hs in hotel_section_from_html:
        a = hs.select_one('p.area')
        if (a != None):
            hotel_section.append(hs) # aに情報がある時に実行したいので、条件はNoneではないというa != Noneになります。

    hotelName = [] # ホテル名を格納する空配列を用意します。
    hotelMinCharge = [] # ホテルの料金を格納する空配列を用意します。
    reviewAverage = [] # ホテル評価を格納する空配列を用意します。
    hotel_locate = [] # ホテル住所を格納する空配列を用意します。

    # hotel_sectionからsectionを1つずつ取り出してhsに代入して実行します。
    for hs in hotel_section:
        hs1 = hs.select_one('h1 a').text #ホテル名をhs1に代入
        if (hs.select_one('p.htlPrice span') != None):
            hs2 = hs.select_one('p.htlPrice span').text.replace("円〜","").replace(",","") # 値段をhs2に代入
        else:
            hs2 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。
        hs3 = hs.select_one('p.cstmrEvl strong').text # 評価をhs3に代入
        hs4 = hs.select_one('p.htlAccess').text.replace("\n","").replace(" ","").replace("[地図を見る]","").replace("　","") # 住所をhs4に代入
        hotelName.append(hs1) #抽出したホテル名をhotelNameに追加
        hotelMinCharge.append(hs2) #抽出したホテルの料金をhotelMinChargeに追加
        reviewAverage.append(hs3) #抽出したホテルの評価をreviewAverageに追加
        hotel_locate.append(hs4) #抽出したホテル住所をhotel_locateに追加

    # pandasのデータフレームに使うデータを定義します。
    data_list = {
        "hotelName" : hotelName,
        "hotelMinCharge" : hotelMinCharge,
        "reviewAverage" : reviewAverage,
        "hotel_locate" : hotel_locate,
    }

    # 定義したデータをpandasに読み込ませます
    # pd.DataFrame(ここにデータ)でデータフレームに変換できます。
    df = pd.DataFrame(data_list)

    # drop_duplicates()で重複したデータを削除してくれます。
    # drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
    df.drop_duplicates(inplace=True)

    # 列の認識番号であるindexが列の順番と一致していないので、reset_indexで番号を振り直します。
    # reset_index(drop=True,inplace=True)とすることで、元のindexを削除して新たに生成したindexを作成し、元のデータを更新してくれます。
    df.reset_index(drop=True,inplace=True)

    # 認証のために機能役割を決めるアクセス先をscopesに設定
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]

    # その役割の許可をもらうAPIキーをservice_account.jsonから読み込み、credentialsに代入
    # 認証キーを使うアクセス先をscopesに代入
    credentials = Credentials.from_service_account_file(
        'service_account.json',
        scopes=scopes
    )

    # 認証情報を格納しているcredentialsを使って、gspread.authorizeでスプレッドシートの使用許可を取り、その認証結果をgcに代入
    gc = gspread.authorize(credentials)

    # 使用するスプレッドシートのアクセス先をSP_SHEET_KEYに代入
    # https://docs.google.com/spreadsheets/d/「ここの部分がSP_SHEET_KEYに代入される」
    SP_SHEET_KEY = '1a-lk8e9ZdDt23qkvvd4UCMNJh6nuLV3IH1bvhnS5x24'

    # 開きたいスプレッドシートを認証結果を格納したgcを使ってgc.open_by_keyで開く
    sh = gc.open_by_key(SP_SHEET_KEY)

    # 参照するシート名をSP_SHEETに代入
    SP_SHEET = 'sample'

    # gc.open_by_keyで開いたスプレッドシートのsampleシートをsh.worksheet(SP_SHEET)で情報を得て、worksheetに代入する
    worksheet = sh.worksheet(SP_SHEET)

    data = worksheet.get_all_values() # スプレッドシートにある既存のデータをdataに代入
    df_old = pd.DataFrame(data[1:], columns=data[0]) # スプレッドシートにある既存のデータをデータフレームに格納し、df_oldに代入
    df_new = df # スクレイピングで取得した新しいデータdfをdf_newに代入
    df_upload = pd.concat([df_old,df_new]) # 既存のdf_oldとdf_newをpd.concatで結合
    df_upload.reset_index(drop=True,inplace=True)

    # シートにアクセス準備が出来たので、set_with_dataframe(どこに,どのデータ,データフレームで自動生成されるindex数字を含むかどうか)
    # を使ってシートにデータフレームのデータを書き込みます。
    set_with_dataframe(sh.worksheet("sample"), df_upload, include_index=False)


schedule.clear() # scheduleに実行予定が残っていないように初期化しておきます。
schedule.every(2).seconds.do(job) # scheduleに実行予定として２秒ごとにjobという関数を実行を追加します。

# whileで繰り返しscheduleの実行予定を確認します。
while True:
    schedule.run_pending() # scheduleに追加されている実行予定をチェックします
    time.sleep(1) # 重くならないように念のため１秒の猶予を設けます。